# 金融市场情绪专题
PCA

In [2]:
import sys
import os
import cudf  #CUDA计算
import pandas as pd
import numpy as np

# 自行编写的包
sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')
from loader.findata_loader import Loader
from utils.sql import DB

# Stata
from pystata import config

config.init('mp')
# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata
               Love U

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


### 数据提取

In [4]:
os.listdir('/data/DataSets/BW_FInMarket_CSMAR/BW_5_CSMAR/')

['QX_STOCKRATE[DES][csv].txt',
 'QX_TRM1.csv',
 'QX_FUNDDISCOUNTPREMIUM[DES][csv].txt',
 'QX_DIVIDENDPREMIUM.csv',
 'QX_FUNDDISCOUNTPREMIUM1.csv',
 'QX_FUNDDISCOUNTPREMIUM.csv',
 'QX_DIVIDENDPREMIUM[DES][csv].txt',
 'QX_TRM.csv',
 'QX_TRM[DES][csv].txt',
 'QX_STOCKRATE.csv',
 'QX_IPO[DES][csv].txt',
 'QX_IPO.csv']

### 股利溢价

In [3]:
pd.read_csv('/data/DataSets/BW_FInMarket_CSMAR/BW_5_CSMAR/QX_DIVIDENDPREMIUM.csv')

,SgnYear,Dnum,NDnum,DMB,NDMB,PDND,LogPDND
0,1991,1,14,0.6049,4.8106,-4.2057,-0.9005
1,1992,4,72,2.4407,4.5652,-2.1245,-0.2720
2,1993,42,182,2.2759,2.6085,-0.3325,-0.0592
3,1994,160,193,1.3538,1.7117,-0.3578,-0.1019
4,1995,265,116,1.0813,1.1420,-0.0607,-0.0237
5,1996,239,357,1.8867,2.1904,-0.3038,-0.0648
6,1997,215,599,2.1294,2.4667,-0.3373,-0.0639
7,1998,283,638,2.0322,2.2689,-0.2367,-0.0479
8,1999,304,719,2.2116,2.4395,-0.2278,-0.0426
9,2000,353,818,2.9219,3.6668,-0.7449,-0.0986


### 股票融资占股票债券融资比例

In [5]:
pd.read_csv('/data/DataSets/BW_FInMarket_CSMAR/BW_5_CSMAR/QX_STOCKRATE.csv')

,SgnYear,IPO,APublic,APrivate,Rs,Refinance,Ashares,Bond,SRate
0,1990,5.86,0.00,0.00,0.00,0.00,5.86,93.49,5.90
1,1991,2.39,2.48,0.00,0.00,2.48,4.87,199.09,2.39
2,1992,153.42,0.00,0.00,0.00,0.00,153.42,395.64,27.94
3,1993,370.50,0.00,0.00,1.95,1.95,372.45,304.54,55.02
4,1994,186.47,7.68,0.00,12.29,19.97,206.44,418.86,33.01
5,1995,25.79,0.00,0.00,25.41,25.41,51.20,376.89,11.96
6,1996,224.91,0.00,0.00,56.17,56.17,281.08,1883.26,12.99
7,1997,618.39,0.00,0.00,186.96,186.96,805.35,835.22,49.09
8,1998,405.32,50.54,0.00,327.44,377.98,783.30,104.10,88.27
9,1999,505.03,55.10,4.65,319.62,379.37,884.40,424.10,67.59


### 基金折溢价率
Symbol [基金代码] -
TradingDate [交易日期] -
CategoryID [基金类别] - S0601=股票型基金;S0602=债券型基金;S0603=货币型基金;S0604=混合型基金;S0605=FOF;S0606=股指期货型基金;S0699=其他
FundTypeID [基金运作方式] - S0501=契约型开放式;S0502=契约型封闭式
IsETF [是否ETF] - 1=是；2=否
IsLOF [是否LOF] - 1=是；2=否
IsIndexFund [是否指数基金] - 1=是；2=否
IsInnovative [是否创新型基金] - 1=是；2=否
NAV [基金份额净值] -
AccumulativeNAV [基金份额累计净值] -
ClosePrice [收盘价] -
CovertRate [溢价率(%)] - 计算公式为：（收盘价-基金份额净值）*101/基金份额净值
TurnoverRate [换手率] -

In [6]:

pd.read_csv('/data/DataSets/BW_FInMarket_CSMAR/BW_5_CSMAR/QX_FUNDDISCOUNTPREMIUM.csv')

,Symbol,TradingDate,CategoryID,FundTypeID,IsETF,IsLOF,IsIndexFund,IsInnovative,NAV,AccumulativeNAV,ClosePrice,CovertRate,TurnoverRate
0,150001,2007-09-21,S0601,S0501,2,2,1,1,1.523,1.523,1.454,-4.53,NaN
1,150001,2007-09-24,S0601,S0501,2,2,1,1,1.537,1.537,1.448,-5.79,NaN
2,150001,2007-09-25,S0601,S0501,2,2,1,1,1.501,1.501,1.387,-7.59,NaN
3,150001,2007-09-26,S0601,S0501,2,2,1,1,1.470,1.470,1.329,-9.59,NaN
4,150001,2007-09-27,S0601,S0501,2,2,1,1,1.504,1.504,1.372,-8.78,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,165313,2021-07-23,S0604,S0501,2,1,2,2,3.424,3.424,3.415,-0.26,0.009300
999996,165313,2021-07-26,S0604,S0501,2,1,2,2,3.379,3.379,3.335,-1.30,0.008049
999997,165313,2021-07-27,S0604,S0501,2,1,2,2,3.291,3.291,3.299,0.24,0.008134
999998,165313,2021-07-28,S0604,S0501,2,1,2,2,3.278,3.278,3.255,-0.70,0.006857


### 市场换手率

In [7]:
pd.read_csv('/data/DataSets/BW_FInMarket_CSMAR/BW_5_CSMAR/QX_TRM.csv')

,TradingDate,MarketType,Parameter,Unit,TurnoverRate1,TurnoverRate2
0,1990-12-19,5,1,日,0.00023,0.00062
1,1990-12-19,1,1,日,0.00023,0.00062
2,1990-12-19,31,1,日,0.00023,0.00062
3,1990-12-19,15,1,日,0.00023,0.00062
4,1990-12-19,6,1,日,0.00023,0.00062
...,...,...,...,...,...,...
999995,2022-12-09,37,10,日,0.09454,0.10513
999996,2022-12-09,47,10,日,0.09229,0.10251
999997,2022-12-09,53,10,日,0.09989,0.11220
999998,2022-12-09,63,10,日,0.09767,0.10957


#### IPO股票信息
Symbol [股票代码] -
ListedDate [上市日期] -
ListedYear [上市年份] -
ABSymbol [AB股交叉码] - 指公司同时发行了A和B股的情况
HSymbol [H股交叉码] - 指的是公司同时发行了H股的情况
EstablishDate [公司成立日期] -
IpoDate [首次招股日期] -
CompanyListedDate [公司上市日期] -
IsIPO [是否首次发行] - A=是；B=否
TotalShares [实际发行总量] -
IssuePrice [发行价格] -
CurrencyCode [发行价格币种] -
ClosePrice [上市首日收盘价] -
ReturnRate [上市首日回报率] -
TurnoverRate1 [上市首日换手率(总股本)] -
TurnoverRate2 [上市首日换手率(流通股本)] -
PE [上市首日市盈率] -
PB [上市首日市净率] -

In [8]:
pd.read_csv('/data/DataSets/BW_FInMarket_CSMAR/BW_5_CSMAR/QX_IPO.csv')

,Symbol,ListedDate,ListedYear,ABSymbol,HSymbol,EstablishDate,IpoDate,CompanyListedDate,IsIPO,TotalShares,IssuePrice,CurrencyCode,ClosePrice,ReturnRate,TurnoverRate1,TurnoverRate2,PE,PB
0,1,1991-04-03,1991,NaN,NaN,1987-12-22,1989-03-10,1991-04-03,A,350000.0,40.000,CNY,49.000,0.225000,0.00000,0.00000,NaN,NaN
1,2,1991-01-29,1991,NaN,2202.0,1984-05-30,1988-12-28,1991-01-29,A,28000000.0,1.000,CNY,14.580,13.580000,0.00004,0.00005,NaN,NaN
2,3,1991-07-03,1991,NaN,NaN,1988-11-09,1989-02-28,1991-07-03,A,42716330.0,10.000,CNY,6.300,-0.370000,0.00303,0.00536,NaN,NaN
3,4,1991-01-14,1991,NaN,NaN,1986-05-05,1989-12-23,1991-01-14,A,12500000.0,1.000,CNY,15.980,14.980000,0.00004,0.00010,NaN,NaN
4,5,1990-12-10,1990,NaN,NaN,1987-07-23,1990-03-03,1990-12-10,A,44338500.0,10.000,CNY,18.030,0.803000,NaN,NaN,NaN,24.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5210,900952,1998-05-19,1998,600190.0,NaN,1993-02-09,1998-05-05,1998-05-19,A,100000000.0,0.210,USD,0.222,0.057143,0.01507,0.08439,6.07,1.57
5211,900953,1998-06-24,1998,NaN,NaN,1998-06-19,1998-05-29,1998-06-24,A,240000000.0,0.276,USD,0.274,-0.007246,0.00773,0.02060,10.20,3.07
5212,900955,1999-01-18,1999,NaN,NaN,1999-01-14,1998-12-24,1999-01-18,A,126500000.0,0.280,USD,0.282,0.007143,0.00717,0.02305,6.86,2.99
5213,900956,1999-07-15,1999,NaN,NaN,1999-03-10,1999-06-23,1999-07-15,A,115000000.0,0.198,USD,0.256,0.292929,0.22066,0.48546,6.35,4.27
